# Plot summary generator

In [ ]:
## adding directory to system path --- execute only once
#import sys,os
#os.chdir('C:\\Users\\Rohit Suresh\\Jupyter projects\\Plot summary generator')
#sys.path.append(os.getcwd())
#sys.path

In [139]:
# import necessary packages and libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import sample
import inspect
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
import importlib
import plot_summary_generator_utils
importlib.reload(plot_summary_generator_utils)
from plot_summary_generator_utils import *
import warnings
warnings.filterwarnings('ignore')

<IPython.core.display.Javascript object>

In [26]:
# see that all your functions from utils have been imported
all_functions = inspect.getmembers(plot_summary_generator_utils , inspect.isfunction)
for fn , _ in all_functions: print(str(fn))

IMDb
comic_plot
comic_plot_agg
comic_titles_finder
gcm
get_wordnet_pos
imdb_id_list
lemma
make_comic_titles_list
plot_aggregator_imdb
plot_aggregator_wiki
plot_puller_imdb
plot_puller_wiki
preprocess_corpus_text
sent_tokenize
titles_list
uReq
word_tokenize


# Train embedding matrix
## 1. Obtain plot summaries

Plot summaries will be obtained from Wikipedia. We will use the 'wikipedia' package to operate on wikipedia pages.

In [27]:
filepath = 'corpus_movies_list.xlsx'
imdb_ids = imdb_id_list(filepath)
imdb_ids

0     tt0371746
1     tt0800080
2     tt1228705
3     tt3501632
4     tt0458339
5     tt0848228
6     tt1300854
7     tt1981115
8     tt1843866
9     tt3896198
10    tt2395427
11    tt0478970
12    tt3498820
13    tt1211837
14    tt3896198
15    tt2250912
16    tt3501632
17    tt1825683
18    tt4154756
19    tt5095030
20    tt4154664
21    tt4154796
Name: imdb id, dtype: object

In [28]:
#%time corpus_raw_text = plot_aggregator_imdb(imdb_ids) 

In [29]:
#corpus_raw_text

In [30]:
#%time corpus_tokens = preprocess_corpus_text(corpus_raw_text)

In [31]:
#corpus_tokens[-50:]

In [32]:
# import necessary libraries
from gensim.models import Word2Vec,KeyedVectors

In [33]:
#%time embed = Word2Vec(corpus_tokens, size = 300, min_count = 3, sg = 1, iter=200, negative=10)

In [34]:
#print(embed)

In [35]:
#embed_words = list(embed.wv.vocab)
#print(embed_words[0:25])

In [36]:
#print(embed.wv['thanos'])

In [37]:
#word_vectors = embed.wv

In [38]:
#res = word_vectors.most_similar(positive=['wasp','man'],negative=['woman'])

In [39]:
#res[:3]

In [40]:
#res1 = word_vectors.most_similar(positive=['barton','woman'],negative=['man'])

In [41]:
#res1[:3]

In [42]:
#res2 =word_vectors.most_similar(positive=['thor','tony'],negative=['odin'])

In [43]:
#res2[:3]

In [44]:
#res3 = word_vectors.similar_by_word('peter')

In [45]:
#res3[:3]

In [46]:
cl = pd.read_csv('comic_titles.csv',header=0,encoding = 'unicode_escape')['comic_title']

In [47]:
s = sample(list(cl),1000)
s

['Akira Vol 1 23',
 'Howard the Duck Holiday Special Vol 1 1',
 'Solitaire Vol 1 5',
 'Hulk Vol 2 32',
 'Elektra Vol 3 6',
 'Ultimate Fantastic Four Vol 1 25',
 'Marvel Action: Spider-Man Vol 1 5',
 'New Warriors Vol 4 20',
 'Marvel Super Action Vol 2 32',
 'Astonishing Spider-Man Vol 1 63',
 'Marvel Boy Vol 2 4',
 'Marvel Mystery Handbook 70th Anniversary Special Vol 1 1',
 'Wolverine: Judgment Night Vol 1 1',
 'Thunderstrike Vol 1 10',
 'Rawhide Kid Vol 1 41',
 'Micronauts Vol 2 18',
 'Master of Kung Fu Vol 1 68',
 'Marvel Graphic Novel Vol 1 13',
 'My Girl Pearl Vol 1 1',
 'Moon Knight Vol 1 12',
 'Hawkeye Vol 5 7',
 'Dream Logic Vol 1 1',
 'Star Masters Vol 1 2',
 'Quasar Vol 1 23',
 'Action Force Vol 1 41',
 'Robocop Vol 1 2',
 'Man-Thing Vol 1 19',
 'Fantastic Four Vol 6 9',
 'Peter Parker The Spectacular Spider-Man Vol 1 5',
 'Friendly Neighborhood Spider-Man Vol 1 9',
 'Navy Combat Vol 1 11',
 'Dennis the Menace Vol 1 4',
 'Maximum Security: Dangerous Planet Vol 1 1',
 'Death W

In [48]:
%%notify
#%time comic_plot_agg(s,'corpus_1000_random')

Wall time: 11min 57s


<IPython.core.display.Javascript object>

In [50]:
#with open('corpus_1000_random.txt','r',encoding = 'utf-8') as f: corpus_raw_text = f.read()

In [56]:
%%notify
#%time corpus_tokens = preprocess_corpus_text(corpus_raw_text)

Wall time: 4min 46s


<IPython.core.display.Javascript object>

In [152]:
%%notify
#%time embed = Word2Vec(corpus_tokens, size = 100, min_count = 3, sg = 1, iter=300, negative=10)

<IPython.core.display.Javascript object>

In [104]:
#embed.wv.save_word2vec_format('1000_comic_model')

C:\Users\Rohit Suresh\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [140]:
word_embed = KeyedVectors.load_word2vec_format('1000_comic_model')

In [141]:
word_embed.wv.similar_by_word('johnny')

[('sue', 0.6410468816757202),
 ('reed', 0.5720858573913574),
 ('ben', 0.569328784942627),
 ('supper', 0.5451163053512573),
 ('flame', 0.4908883273601532),
 ('chili', 0.48379915952682495),
 ('subterraneans', 0.4765200614929199),
 ('blackbeard', 0.4531882703304291),
 ('franklin', 0.4512457549571991),
 ('shakti', 0.4510565400123596)]

In [142]:
word_embed.wv.most_similar(positive = ['johnny','woman'], negative = ['man'])

[('invisible', 0.4658660888671875),
 ('walker', 0.4179362654685974),
 ('franklin', 0.4083186984062195),
 ('odds', 0.4004364013671875),
 ('supper', 0.39940154552459717),
 ('sue', 0.3982470631599426),
 ('valeria', 0.3975268602371216),
 ('button', 0.38520193099975586),
 ('reed', 0.38216841220855713),
 ('insists', 0.38033542037010193)]

In [143]:
word_embed.wv.similar_by_word('thanos')

[('acquire', 0.6423979997634888),
 ('champion', 0.46437862515449524),
 ('13', 0.4631972014904022),
 ('captor', 0.4537425935268402),
 ('await', 0.4433976411819458),
 ('preparation', 0.43442627787590027),
 ('infinity', 0.43178561329841614),
 ('havoc', 0.43136969208717346),
 ('prize', 0.4304622709751129),
 ('thane', 0.4285545349121094)]

In [144]:
word_embed.wv.similar_by_word('galactus')

[('incursion', 0.5404379367828369),
 ('surfer', 0.5186083316802979),
 ('consume', 0.48616886138916016),
 ('builder', 0.4852069020271301),
 ('devourer', 0.4683842658996582),
 ('caiera', 0.45831814408302307),
 ('richards', 0.4549655318260193),
 ('illuminati', 0.44175058603286743),
 ('earth8009', 0.43434104323387146),
 ('asgard', 0.42935246229171753)]

In [145]:
word_embed.wv.similar_by_word('thor')

[('mjolnir', 0.7003511190414429),
 ('asgard', 0.5992928743362427),
 ('freya', 0.5697595477104187),
 ('chucky', 0.5385968089103699),
 ('olympus', 0.532562792301178),
 ('hercules', 0.5038235187530518),
 ('hammer', 0.5016287565231323),
 ('hela', 0.4937015175819397),
 ('jakes', 0.4813027083873749),
 ('thrym', 0.4778319299221039)]

In [146]:
word_embed.wv.similar_by_word('asgard')

[('odin', 0.6834082007408142),
 ('thor', 0.5992928743362427),
 ('majeston', 0.5764435529708862),
 ('zelia', 0.5747852325439453),
 ('adva', 0.5327659845352173),
 ('gateway', 0.5269265174865723),
 ('olympus', 0.5100908279418945),
 ('manipulates', 0.5028811693191528),
 ('valhalla', 0.4570693373680115),
 ('perrikus', 0.45525404810905457)]

In [147]:
word_embed.wv.similar_by_word('loki')

[('freya', 0.5285950899124146),
 ('tesseract', 0.47756025195121765),
 ('surfer', 0.4680214524269104),
 ('warp', 0.4569333493709564),
 ('asgard', 0.44708818197250366),
 ('mjolnir', 0.43616780638694763),
 ('emma', 0.4324970841407776),
 ('romanoff', 0.42658719420433044),
 ('earth8009', 0.42630529403686523),
 ('chariot', 0.42450594902038574)]

In [148]:
word_embed.wv.most_similar(positive = ['thor','beta','ray','bill'], negative = ['mjolnir'])

[('sif', 0.6179021000862122),
 ('unlimiteds', 0.5002724528312683),
 ('tester', 0.4595463275909424),
 ('conclude', 0.4520714581012726),
 ('harrison', 0.4496876299381256),
 ('aloysius', 0.4433065354824066),
 ('motherboard', 0.44249773025512695),
 ('barkeep', 0.4384458363056183),
 ('kenji', 0.4348958134651184),
 ('n', 0.4263717830181122)]

In [149]:
word_embed.wv.most_similar(positive = ['thor','loki'], negative = ['odin'])

[('mjolnir', 0.5143409967422485),
 ('freya', 0.5035570859909058),
 ('chucky', 0.46795350313186646),
 ('olympus', 0.38845759630203247),
 ('deem', 0.37002259492874146),
 ('resigns', 0.3641202449798584),
 ('overpower', 0.3625503182411194),
 ('tesseract', 0.3613758087158203),
 ('self', 0.35894039273262024),
 ('need', 0.3585434854030609)]

In [150]:
word_embed.wv.most_similar(positive = ['clint','woman'], negative = ['man'])

[('mobius', 0.4300547242164612),
 ('kale', 0.4289330840110779),
 ('barton', 0.42411309480667114),
 ('morris', 0.41994786262512207),
 ('spends', 0.41976305842399597),
 ('odds', 0.411861777305603),
 ('biological', 0.4104948043823242),
 ('recollection', 0.40786492824554443),
 ('dad', 0.39982008934020996),
 ('predator', 0.39492446184158325)]

In [151]:
word_embed.wv.similar_by_word('namor')

[('submariner', 0.6122113466262817),
 ('atlantis', 0.511634111404419),
 ('warlord', 0.4946136474609375),
 ('scarrs', 0.48963773250579834),
 ('lemuria', 0.47273361682891846),
 ('torpedo', 0.47254741191864014),
 ('krang', 0.46406590938568115),
 ('munson', 0.46191802620887756),
 ('sub', 0.46036002039909363),
 ('shark', 0.45511379837989807)]